### Import Pagckages/Create Def

In [1]:
import pandas as pd
import geopandas as gpd
import os
from math import radians, cos, sin, asin, sqrt, atan2
import pyodbc
import numpy as np
from shapely.geometry import Polygon, MultiPoint, Point
from shapely.ops import nearest_points
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.9.0-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result in KM
    return(c * r)

### Load Datasets
Required
- Tabblock Centriod
- Population 
- Poltical Geogrpahy

In [3]:
os.chdir()
gp_CD = gpd.read_file()

In [6]:
#Import target level pop data
conn = pyodbc.connect()
df_blocks_data = pd.read_sql_query(f"""SELECT GEOCODE AS GEOID20
      ,CAST([P0010004] as INT) AS POP100
      ,CAST([INTPTLAT] as FLOAT) AS INTPTLAT
      ,CAST([INTPTLON] AS FLOAT) AS INTPTLON
  FROM [vw].[PL2020_BLK] WHERE [STUSAB] = 'IL'""",conn)

In [7]:
#Import Poltical GEO / BAF
pd_joiner = pd.read_csv()

### Reformat/Build Datasets

In [8]:
gp_CD_cols = [i for i in gp_CD.columns if i not in ['GEOID20','INTPTLAT20','INTPTLON20','geometry']]
gp_CD = gp_CD.drop(columns=gp_CD_cols)
gp_CD.head()

,GEOID20,INTPTLAT20,INTPTLON20,geometry
0,1701,+41.5466708,-087.8357472,"POLYGON ((-88.13677 41.42285, -88.13633 41.422..."
1,1718,+40.2271844,-090.0691183,"POLYGON ((-91.51297 40.18106, -91.51283 40.181..."
2,1707,+41.8633320,-087.7321218,"POLYGON ((-87.91961 41.88220, -87.91960 41.882..."
3,1702,+41.2562081,-087.7901148,"POLYGON ((-88.25150 41.11422, -88.25144 41.114..."
4,1714,+42.0255473,-088.4458531,"POLYGON ((-88.71978 42.01944, -88.71935 42.019..."


In [9]:
df_pl94 = gpd.GeoDataFrame(df_blocks_data, geometry=gpd.points_from_xy(df_blocks_data.INTPTLON ,df_blocks_data.INTPTLAT))

In [10]:
pd_joiner_cols = [i for i in pd_joiner.columns if i not in ['GEOID20','CD116_GEOID']]
pd_joiner = pd_joiner.drop(columns=pd_joiner_cols)

In [11]:
df_tabblock20_CDFP116 = df_pl94.merge(pd_joiner, on='GEOID20')

### Population From Center
This test uses a population centroid the boundary or the political district and the geographic centroid as a preliminary test to measure the distance from the center and exterior of a political district. This test is not to state whether a district is gerrymandered but to help fill out our understanding of the district compactness

The messure takes the population center and the distance from the boundry and geography center to create a ratio between 0 and 1 with more compact districts being closer to 0

In [12]:
dfList = list(set(df_tabblock20_CDFP116['CD116_GEOID'].tolist()))
cdObj = {i: {'wx': None, 'wy':None, 'cx':None , 'cy':None} for i in  dfList}
for cd in dfList:
    df_cd = df_tabblock20_CDFP116[df_tabblock20_CDFP116['CD116_GEOID'] == cd]
    m = df_cd['POP100'].sum()
    vx = 0
    vy = 0 
    for row in df_cd.iterrows():
        vy += row[1]['POP100']*row[1]['INTPTLAT'] 
        vx += row[1]['POP100']*row[1]['INTPTLON'] 
    cdObj[cd]['wy'] = vy/m
    cdObj[cd]['wx'] = vx/m
    cd_geo = gp_CD[gp_CD['GEOID20'] == cd]
    cdObj[cd]['cy'] =  cd_geo.iloc[0]['INTPTLAT20']
    cdObj[cd]['cx'] =  cd_geo.iloc[0]['INTPTLON20']
df_centernoids = pd.DataFrame.from_dict(cdObj, orient='index')

<ipython-input-12-36c397a2e0d0>:11: RuntimeWarning: invalid value encountered in true_divide
  cdObj[cd]['wy'] = vy/m
<ipython-input-12-36c397a2e0d0>:12: RuntimeWarning: invalid value encountered in true_divide
  cdObj[cd]['wx'] = vx/m


In [13]:
df_centernoids = df_centernoids.dropna(how='any')
dfList = list(df_centernoids.index)
df_centernoids.to_csv('basic_points.csv')
df_centernoids.head(len(df_centernoids.index))

,wx,wy,cx,cy
1707,-87.722353,41.860745,-087.7321218,+41.8633320
1708,-88.120427,42.002132,-088.0967252,+42.0093710
1714,-88.281109,41.908338,-088.4458531,+42.0255473
1701,-87.643575,41.736597,-087.8357472,+41.5466708
1718,-89.802074,40.307093,-090.0691183,+40.2271844
1713,-89.031085,39.873223,-089.5198424,+39.5156354
1710,-87.890034,42.339133,-087.9399419,+42.2800147
1712,-89.906728,38.456525,-089.4278909,+37.9933075
1709,-87.692369,42.009767,-087.8101853,+42.0526844
1717,-89.694317,41.483692,-090.2186532,+41.3894669


In [14]:
df_wp = df_centernoids[['wy','wx']]
df_wp = gpd.GeoDataFrame(df_wp, geometry = gpd.points_from_xy(df_wp.wx,df_wp.wy))
df_cp = df_centernoids[['cy','cx']]
df_cp = gpd.GeoDataFrame(df_cp, geometry = gpd.points_from_xy(df_cp.cx,df_cp.cy))
df_cp = df_cp.set_crs('epsg:4269')
df_wp = df_wp.set_crs('epsg:4269')

In [15]:
district_profile = {i: {'pop_center':None,'pop_boundry':None} for i in  dfList}

In [16]:
for dis in dfList:
    df_c = df_cp[df_cp.index==dis]
    df_w = df_wp[df_wp.index==dis]
    district_profile[dis]['pop_center'] = distance(df_c['cy'],df_w['wy'],df_c['cx'],df_w['wx'])

In [17]:
for dis in dfList:
    gp_CD_init = gp_CD[gp_CD['GEOID20']==dis]
    df_wp_dis = df_wp[df_wp.index==dis]
    points2 = gp_CD_init.copy()
    points2 = points2.to_crs('epsg:4269')
    points2['geometry'] = points2.exterior
    nearest_geoms = nearest_points(df_wp_dis.unary_union,points2.unary_union)
    ear_idx0 = nearest_geoms[0]
    ear_idx1 = nearest_geoms[1]
    district_profile[dis]['pop_boundry'] = distance(ear_idx0.y,ear_idx1.y,ear_idx0.x,ear_idx1.x)
    
df_pointDistance = pd.DataFrame.from_dict(district_profile, orient='index')
df_pointDistance['total_distance'] = df_pointDistance['pop_center']+df_pointDistance['pop_boundry']
df_pointDistance['pop_compact'] = df_pointDistance['pop_center']/df_pointDistance['total_distance']

In [18]:
df_pointDistance.to_csv('base_distance.csv')
df_pointDistance.head(len(df_pointDistance.index))

,pop_center,pop_boundry,total_distance,pop_compact
1707,0.858633,1.233582,2.092215,0.410394
1708,2.117384,4.438609,6.555994,0.322969
1714,18.851531,0.553470,19.405001,0.971478
1701,26.476648,2.509163,28.985811,0.913435
1718,24.337591,17.631005,41.968596,0.579900
1713,57.703946,10.783898,68.487843,0.842543
1710,7.749515,2.313123,10.062639,0.770128
1712,66.351899,22.331290,88.683189,0.748190
1709,10.838032,2.144285,12.982316,0.834830
1717,44.947612,11.226406,56.174018,0.800149


### Kmean Symmetry 
Using Kmean Clustering we create 2 groups using target communtiy population to find compact groups. We then take the Population Center test to create compartive metrics where the closer the 2 groups results are the more compact it is

In [ ]:
simp_set_all = df_tabblock20_CDFP116.loc[:, ['GEOID20','POP100','INTPTLAT','INTPTLON','CD116_GEOID']]
df_kcluster = pd.DataFrame(columns=['GEOID20','POP100','INTPTLAT','INTPTLON','CD116_GEOID','Cluster_label'])
for cd in dfList:
    simp = simp_set_all[simp_set_all['CD116_GEOID']==cd]
    kmeans = KMeans(n_clusters = 2, n_init=50,max_iter=1000, init ='k-means++')
    lat_long = simp[simp.columns[3:4]]
    blk_size = simp[simp.columns[1]]
    simp_kmean_clusters = kmeans.fit(lat_long,sample_weight=blk_size)
    simp['Cluster_label'] = kmeans.predict(lat_long,sample_weight=blk_size)
    simp['Cluster_label'] = simp['Cluster_label']+1
    df_kcluster = df_kcluster.append(simp)

In [ ]:
clusterN = [1,2]
kcObj = {i: {'1':{},'2':{}} for i in  dfList}
for cd in dfList:
    for clu in clusterN:
        df_k = df_kcluster[(df_kcluster['CD116_GEOID']==cd) & (df_kcluster['Cluster_label']==clu)]
        m = df_k['POP100'].sum()
        vx =0
        vy = 0 
        for row in df_k.iterrows():
            vy += row[1]['POP100']*row[1]['INTPTLAT'] 
            vx += row[1]['POP100']*row[1]['INTPTLON']
        df_k_geo = gpd.GeoDataFrame(df_k, geometry=gpd.points_from_xy(df_k.INTPTLON, df_k.INTPTLAT))
        poly = Polygon([p for p in  df_k_geo['geometry'].tolist()])
        kcObj[cd][str(clu)]['cx']= poly.centroid.x
        kcObj[cd][str(clu)]['cy']= poly.centroid.y
        kcObj[cd][str(clu)]['wy']= vy/m
        kcObj[cd][str(clu)]['wx']= vx/m

In [ ]:
dfKcc = pd.DataFrame(columns=['CD','Cluster','Center_Point','Weighted_Point'])
for k,v in kcObj.items():
    for i,j in v.items():
        dfKcc = dfKcc.append({'CD':k,'Cluster':i,'Center_Point':Point(j['cx'],j['cy']),'Weighted_Point':Point(j['wx'],j['wy'])},ignore_index=True)

In [ ]:
dfKcc.to_csv('kmean_point.csv')
dfKcc.head()
dfKcc = dfKcc.drop(columns='Center_Point')
dfKcc = dfKcc.rename(columns={'CD':'GEOID20'})

In [ ]:
dfKcc = gpd.GeoDataFrame(dfKcc, geometry='Weighted_Point')

In [ ]:
dfKcc = dfKcc.set_crs('epsg:4269')

In [ ]:
df_sgerry = gpd.sjoin(dfKcc, gp_CD, lsuffix='_Cluster',rsuffix='_SHP')

In [ ]:
df_sgerry = df_sgerry.drop(columns=['index__SHP','INTPTLAT20','INTPTLON20','Weighted_Point'])

In [ ]:
df_sgerry['gerry_check'] = False
for i, row in df_sgerry.iterrows():
    if row['GEOID20__Cluster'] != row['GEOID20__SHP']:
        df_sgerry.at[i, 'gerry_check'] = True
df_sgerry_check = df_sgerry[df_sgerry['gerry_check']==True]       

In [ ]:
df_sgerry_check = df_sgerry_check.sort_values(by=['GEOID20__Cluster']).reset_index(drop=True)
df_sgerry_check.head(len(df_sgerry_check.index))